In [1]:
import collections
import textwrap
import typing as tp

In [2]:
MAX_STEPS = 250

In [3]:
class Grammar:
    def __init__(
        self,
        rules: tp.Mapping[str, tp.Sequence[str]],
        *,
        nonterminals: tp.Optional[tp.Sequence[str]] = None,
    ) -> None:
        self.rules = {key: list(value) for key, value in rules.items()}
        self.nonterminals = set(nonterminals or self.rules)

    def is_symbol_nonterminal(self, symbol: str) -> bool:
        return symbol in self.nonterminals

    def is_symbol_terminal(self, symbol: str) -> bool:
        return not self.is_symbol_nonterminal(symbol)

    def is_word_terminal(self, word: str) -> bool:
        return all(map(self.is_symbol_terminal, word))

    def generate_next(self, word: str) -> tp.Generator[str, None, None]:
        for i, char in enumerate(word):
            if self.is_symbol_nonterminal(char):
                for production in self.rules.get(char, []):
                    yield word[:i] + production + word[i + 1:]
                break


In [4]:
def generate_bfs_paths(
    grammar: Grammar,
    start: str,
    max_steps: int = 15,
) -> tp.Generator[tuple[str, list[str]], None, None]:
    queue = collections.deque([(start, [start])])

    while queue:
        current_word, path = queue.popleft()
        yield current_word, path

        if len(path) >= max_steps:
            continue

        for nxt in grammar.generate_next(current_word):
            new_path = path + [nxt]
            queue.append((nxt, new_path))

In [5]:
def find_derivation_path(
    grammar: Grammar,
    start: str,
    target: str,
    max_steps: int = 15,
) -> tp.Optional[list[str]]:
    for current_word, path in generate_bfs_paths(grammar, start, max_steps):
        if current_word == target:
            return path

    return None

In [6]:
# Has to be optimized
def find_all_terminals(
    grammar: Grammar,
    start: str,
    max_steps: int = 15,
) -> set[str]:
    results = set()

    for current_word, _ in generate_bfs_paths(grammar, start, max_steps):
        if grammar.is_word_terminal(current_word):
            results.add(current_word)

    return results

In [7]:
def find_all_terminal_derivations(
    grammar: Grammar,
    start: str,
    max_steps: int = 15,
) -> dict[str, list[list[str]]]:
    derivations = {}

    for current_word, path in generate_bfs_paths(grammar, start, max_steps):
        if grammar.is_word_terminal(current_word):
            derivations.setdefault(current_word, []).append(path)
    return derivations

In [8]:
def print_derivation_path(derivation: tp.Optional[tp.Sequence[str]]) -> None:
    if derivation:
        print("Найден вывод:")
        for step in derivation:
            print(step)
    else:
        print("Вывод целевой строки не найден в пределах заданного числа шагов.")

In [9]:
def indent_text(text: str, indent: int) -> str:
    return textwrap.indent(text, " " * indent)

### Задание 1

In [10]:
rules = {
    "A": ["aAa", "a"],
    "B": ["bBb", "a"],
}

In [11]:
grammar = Grammar(rules)

In [12]:
start = "AbB"
target = "aaabb"

In [13]:
print_derivation_path(
    find_derivation_path(
        grammar,
        start,
        target,
        max_steps=MAX_STEPS,
    ),
)

Вывод целевой строки не найден в пределах заданного числа шагов.


### Задание 2

In [14]:
rules = {
    "S": ["aAa", "bAb", "aa", "bb"],
    "A": ["aAa", "bAb", "aa", "bb"],
    "F": [],
}

In [15]:
grammar = Grammar(rules)

In [16]:
start = "S"
target = "aaaabbbbaaaa"

In [17]:
print_derivation_path(
    find_derivation_path(
        grammar,
        start,
        target,
        max_steps=MAX_STEPS,
    ),
)

Найден вывод:
S
aAa
aaAaa
aaaAaaa
aaaaAaaaa
aaaabAbaaaa
aaaabbbbaaaa


### Задание 5

In [18]:
rules = {
    "S": ["bA", "aB"],
    "A": ["a", "aS", "bAA"],
    "B": ["b", "bS", "aBB"],
}

In [19]:
start = "S"

In [20]:
grammar = Grammar(rules)

In [ ]:
terminal_strings = find_all_terminals(grammar, start, max_steps=20)

In [20]:
print(len(terminal_strings))
balanced = all(string.count("a") == string.count("b") for string in terminal_strings)
print("Все цепочки сбалансированы?", balanced)

250952
Все цепочки сбалансированы? True


### Задание 6

In [21]:
all_derivations = find_all_terminal_derivations(grammar, start, max_steps=7)

In [22]:
ambiguous = {w: paths for w, paths in all_derivations.items() if len(paths) > 1}

In [23]:
if ambiguous:
    print("Найдены неоднозначные цепочки:", end="\n\n")

    for w, paths in ambiguous.items():
        print(f"Строка: {w}")
        print("Количество различных выводов:", len(paths))
        for i, path in enumerate(paths, 1):
            print(indent_text(f"Вывод {i}:", 1))
            for step in path:
                print(indent_text(step, 2))
        print()
else:
    print("Не обнаружено неоднозначности в пределах заданного числа шагов.")

Найдены неоднозначные цепочки:

Строка: bbaaba
Количество различных выводов: 2
 Вывод 1:
  S
  bA
  bbAA
  bbaA
  bbaaS
  bbaabA
  bbaaba
 Вывод 2:
  S
  bA
  bbAA
  bbaSA
  bbaaBA
  bbaabA
  bbaaba

Строка: bbabaa
Количество различных выводов: 2
 Вывод 1:
  S
  bA
  bbAA
  bbaA
  bbabAA
  bbabaA
  bbabaa
 Вывод 2:
  S
  bA
  bbAA
  bbaSA
  bbabAA
  bbabaA
  bbabaa

Строка: aabbab
Количество различных выводов: 2
 Вывод 1:
  S
  aB
  aaBB
  aabB
  aabbS
  aabbaB
  aabbab
 Вывод 2:
  S
  aB
  aaBB
  aabSB
  aabbAB
  aabbaB
  aabbab

Строка: aababb
Количество различных выводов: 2
 Вывод 1:
  S
  aB
  aaBB
  aabB
  aabaBB
  aababB
  aababb
 Вывод 2:
  S
  aB
  aaBB
  aabSB
  aabaBB
  aababB
  aababb

